In [1]:
import torch
from tensorflow.python.ops.ragged.ragged_util import lengths_to_splits

from classes import FullFileForTestings  # Import dataset class and model
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from classes import SpeedEstimatorRNNModified, SpeedEstimatorLSTMModified
import numpy as np

In [2]:
#Hyperparameters
input_size = 20  # Number of CAN signals per timestep
hidden_size = [256]
num_layers = [3]
output_size = 2

In [3]:
num_models = 1
num_test_data_path = ["data/i7/it_1/3_validation/i7_8.csv", "data/i7/it_1/3_validation/i7_16.csv",
 "data/i7/it_1/3_validation/i7_31.csv", "data/i7/it_1/3_validation/i7_37.csv", "data/i7/it_1/3_validation/i7_43.csv", "data/i7/it_1/3_validation/i7_49.csv"]
test_data = ["suay_steer_ON", "suay_steer_OFF", "auax_drv_OFF", "buax_brk_ON", "buax_brk_OFF", "sin_steer_ON"]
model_location = "LSTM/trained_models/i7/it_1/state_models/model_LSTM_"

plot_location = "LSTM/trained_models/i7/it_1/validation_plots/model_LSTM_"

In [5]:
for i in range(num_models):
    # Set paths and parameters for testing
    model_path = model_location + str(i) + '.pt'  # Replace with the path to your saved model

    for j in range(len(num_test_data_path)):
        test_csv_path = num_test_data_path[j]
        # Prepare the test dataset
        test_dataset = FullFileForTestings(test_csv_path)
        features, actual_speeds = test_dataset.get_full_data()

        # Pass all data to the model directly
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        features = features.unsqueeze(0)  # Add batch dimension
        features = features.to(device)

        # Define and load the model
        model = SpeedEstimatorLSTMModified(input_size, hidden_size[i], num_layers[i], output_size)  # Instantiate your LSTM model
        checkpoint = torch.load(model_path, map_location=device)

        model.load_state_dict(checkpoint['model_state_dict'])  # Load the trained weights
        model = model.to(device)  # Move model to the appropriate device
        model.eval()  # Set model to evaluation model

        # Inference
        with torch.no_grad():
            predictions = model(features)
            predicted_speeds = predictions.cpu().numpy().flatten()

        length = len(predicted_speeds)/2
        u = [0]*int(length)
        v = [0]*int(length)

        for k in range(len(predicted_speeds)):
            if k % 2 == 0:
                u[int(k/2)] = predicted_speeds[k]
            else:
                v[int((k-1)/2)] = predicted_speeds[k]

        # longitudinal plots
        plt.figure(figsize=(10, 6))
        plt.plot(actual_speeds[:, 0], label="Actual Speed")
        plt.plot(u, label="Estimated Speed", linestyle="dashed")
        plt.legend()
        plt.title("Comparison of Actual vs. Predicted Longitudinal Speed (" + test_data[j] + ")")
        plt.xlabel("Time Step [s]")
        plt.ylabel("Speed [m/s]")
        plt.grid()
        plt.savefig(plot_location + str(i) + "/u/model_LSTM_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_u.png", dpi=300,)
        plt.close()

        plt.figure(figsize=(10, 6))
        plt.plot(actual_speeds[:, 0].numpy() - u, label="Difference")
        plt.legend()
        plt.title("Difference of Actual vs. Predicted Longitudinal Speed (" + test_data[j] + ")")
        plt.xlabel("Time Step [s]")
        plt.ylabel("Speed difference [m/s]")
        plt.grid()
        plt.savefig(plot_location + str(i) + "/u/model_LSTM_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_u_diff.png", dpi=300)
        plt.close()

        # lateral plots
        plt.figure(figsize=(10, 6))
        plt.plot(actual_speeds[:, 1], label="Actual Speed")
        plt.plot(v, label="Estimated Speed", linestyle="dashed")
        plt.legend()
        plt.title("Comparison of Actual vs. Predicted Lateral Speed (" + test_data[j] + ")")
        plt.xlabel("Time Step [s]")
        plt.ylabel("Speed [m/s]")
        plt.grid()
        plt.savefig(plot_location + str(i) + "/v/model_LSTM_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_v.png", dpi=300,)
        plt.close()

        plt.figure(figsize=(10, 6))
        plt.plot(actual_speeds[:, 1].numpy() - v, label="Difference")
        plt.legend()
        plt.title("Difference of Actual vs. Predicted Lateral Speed (" + test_data[j] + ")")
        plt.xlabel("Time Step [s]")
        plt.ylabel("Speed difference [m/s]")
        plt.grid()
        plt.savefig(plot_location + str(i) + "/v/model_LSTM_" + str(i) + "_" + test_data[j] + "_act_vs_predicted_v_diff.png", dpi=300)
        plt.close()

------------------------------------------------------------------------------------------------
debugging